In [1]:
import psycopg2 as pg

class Db_deputados():
    def __init__(self, nome_db: str) -> None:
        self.nome_db = nome_db
        
        conn = pg.connect(host='localhost',
                            user='postgres',
                            password='postgres',
                            port=5432)
        conn.autocommit = True

        cursor = conn.cursor()

        cursor.execute("DROP DATABASE " + self.nome_db)

        conn.close()

    def connect(self) -> pg:
        connection = pg.connect(host='localhost',
                                database=self.nome_db,
                                user='postgres',
                                password='postgres')
        
        return connection
    
    def cria_db(self):
        conn = pg.connect(host='localhost',
                            user='postgres',
                            password='postgres',
                            port=5432)
        conn.autocommit = True

        cursor = conn.cursor()

        cursor.execute("CREATE DATABASE " + self.nome_db)

        conn.close()

    def run_sql(self, sql: str) -> None:        
        connection = self.connect() # cria a conexao com o BD
        cursor = connection.cursor() # permite que o python execute o o comando em SQL no BD

        cursor.execute(sql) # executa o comando

        connection.commit() # confirma todas as alteracoes feitas

        connection.close()  # fecha a conexao
    
    def copy_csv(self, sql):
        connection = self.connect()
        cursor = connection.cursor()

        cursor.execute(sql)

        connection.commit() 
        connection.close()

    def consultas(self, sql: str) -> list:
        conn = self.connect()
        conn.autocommit = True

        cursor = conn.cursor()

        cursor.execute(sql)

        result = cursor.fetchall()
        
        return result

In [2]:
db = Db_deputados(nome_db="deputados")

db.cria_db()

# exclui todas as tabelas se elas existirem
sql = '''DROP TABLE IF EXISTS Deputados;DROP TABLE IF EXISTS Partidos;'''
db.run_sql(sql)

In [3]:
# cria a tabela Partidos
sql = '''CREATE TABLE IF NOT EXISTS Partidos (
	        id varchar(2) NOT NULL PRIMARY KEY,
	        nome varchar(14) NOT NULL
        );'''
db.run_sql(sql)

In [4]:
# cria a tabela Deputados
sql = '''CREATE TABLE IF NOT EXISTS Deputados (
            id varchar(7) NOT NULL PRIMARY KEY,
            nome varchar(150) NOT NULL,
            partidoId varchar(2) NOT NULL
                REFERENCES Partidos(id),
            siglaUf char(2) NOT NULL,
            idLegislatura varchar(3)
        );'''
db.run_sql(sql)

In [5]:
# povoando as tabelas

# file_partidos = open("partidos.csv", "r")
# file_deputados = open("deputados.csv", "r")

# # tabela partidos
# db.copy_csv(file_partidos, "Partidos")

sql = '''COPY Partidos(id, nome) FROM
        '/home/dicati/Documentos/IFMS/2022.2/SECOMP/manipulacao/partidos.csv' csv 
        header delimiter ',';'''
db.run_sql(sql)


In [6]:
# db.copy_csv(file_deputados, "Deputados")

sql = '''COPY Deputados(id, nome, partidoId, siglaUf, idLegislatura) FROM
      '/home/dicati/Documentos/IFMS/2022.2/SECOMP/manipulacao/deputados.csv' csv 
      header delimiter ',';'''
db.run_sql(sql)

In [7]:
import pandas as pd

consulta_sql = '''SELECT * FROM Partidos LIMIT 5;'''

resultado = db.consultas(consulta_sql)

print(resultado)

consulta_sql = '''SELECT * FROM deputados LIMIT 8;'''

resultado = db.consultas(consulta_sql)

print(resultado)

# print(pd.DataFrame(resultado, columns=['id', 'nome', 'idPartido', 'estado', 'idLegislatura']))

[('0', 'AVANTE'), ('1', 'CIDADANIA'), ('2', 'MDB'), ('3', 'NOVO'), ('4', 'PATRIOTA')]
[('204521', 'Abou Anni', '22', 'SP', '56'), ('204379', 'Acácio Favacho', '2', 'AP', '56'), ('204560', 'Adolfo Viana', '14', 'BA', '56'), ('204528', 'Adriana Ventura', '3', 'SP', '56'), ('121948', 'Adriano do Baldy', '9', 'GO', '56'), ('74646', 'Aécio Neves', '14', 'MG', '56'), ('141372', 'Aelton Freitas', '9', 'MG', '56'), ('160508', 'Afonso Florence', '16', 'BA', '56')]


In [8]:
consulta_sql = '''SELECT deputados.nome, partidos.nome FROM Deputados
                JOIN partidos ON partidos.id = deputados.partidoId
                LIMIT 8; '''

resultado = db.consultas(consulta_sql)

resultado

[('Abou Anni', 'UNIÃO'),
 ('Acácio Favacho', 'MDB'),
 ('Adolfo Viana', 'PSDB'),
 ('Adriana Ventura', 'NOVO'),
 ('Adriano do Baldy', 'PP'),
 ('Aécio Neves', 'PSDB'),
 ('Aelton Freitas', 'PP'),
 ('Afonso Florence', 'PT')]

In [9]:
consulta_sql = '''SELECT partidos.nome, COUNT(*) FROM deputados
                JOIN partidos on partidos.id = deputados.partidoid
                GROUP BY partidos.id
                ORDER BY COUNT(*) DESC;'''

resultado = db.consultas(consulta_sql)

resultado

[('PL', 76),
 ('PP', 58),
 ('PT', 56),
 ('UNIÃO', 51),
 ('PSD', 46),
 ('REPUBLICANOS', 44),
 ('MDB', 37),
 ('PSB', 24),
 ('PSDB', 22),
 ('PDT', 19),
 ('PODE', 9),
 ('PCdoB', 8),
 ('PSC', 8),
 ('NOVO', 8),
 ('PSOL', 8),
 ('SOLIDARIEDADE', 8),
 ('CIDADANIA', 7),
 ('AVANTE', 6),
 ('PATRIOTA', 5),
 ('PV', 4),
 ('PROS', 4),
 ('PTB', 3),
 ('REDE', 2)]

In [10]:
consulta_sql = '''SELECT deputados.siglauf, COUNT(*) FROM deputados
                JOIN partidos on partidos.id = deputados.partidoid
                GROUP BY deputados.siglauf
                ORDER BY COUNT(*) ASC;'''

resultado = db.consultas(consulta_sql)

resultado

[('SE', 8),
 ('DF', 8),
 ('RN', 8),
 ('AM', 8),
 ('AP', 8),
 ('TO', 8),
 ('MT', 8),
 ('RR', 8),
 ('AC', 8),
 ('RO', 8),
 ('MS', 8),
 ('AL', 9),
 ('PI', 10),
 ('ES', 10),
 ('PB', 12),
 ('SC', 16),
 ('GO', 17),
 ('PA', 17),
 ('MA', 18),
 ('CE', 22),
 ('PE', 25),
 ('PR', 30),
 ('RS', 31),
 ('BA', 39),
 ('RJ', 46),
 ('MG', 53),
 ('SP', 70)]

In [11]:
novo_dado = '''INSERT INTO deputados(id, nome, partidoid, siglauf)
                VALUES('208080','Vivian Silva', '20', 'MS');'''

db.run_sql(novo_dado)

In [12]:
consulta_sql = '''SELECT deputados.nome, deputados.siglauf, partidos.nome FROM deputados
                join partidos on partidos.id = deputados.partidoid
                WHERE deputados.nome = 'Vivian Silva' '''

resultado = db.consultas(consulta_sql)

resultado

[('Vivian Silva', 'MS', 'REPUBLICANOS')]

In [13]:
exclui_dado = '''DELETE FROM deputados
                WHERE deputados.id = '74646'; '''

db.run_sql(exclui_dado)

In [14]:
consulta_excluido = '''SELECT nome, siglauf FROM deputados
                where id = '74646' '''

resultado = db.consultas(consulta_excluido)

resultado

[]

In [15]:
import plotly as plt
import plotly.express as px
import json
from urllib.request import urlopen
import pandas as pd

In [18]:
geojson = json.load(open("./minicurso/brasil_estados.json"))

In [19]:
consulta_mapa = '''SELECT deputados.siglauf, COUNT(*) FROM deputados
                JOIN partidos on partidos.id = deputados.partidoid
                GROUP BY deputados.siglauf
                ORDER BY COUNT(*) ASC;'''

df_estados = pd.DataFrame(db.consultas(consulta_mapa), columns=['sigla', 'quantidade'])

df_estados

,sigla,quantidade
0,SE,8
1,DF,8
2,RN,8
3,AM,8
4,AP,8
5,TO,8
6,MT,8
7,RR,8
8,AC,8
9,RO,8


In [ ]:
fig = px.choropleth(
    df_estados,
    geojson=geojson,
    locations='sigla',
    color='quantidade',
    color_continuous_scale="blues",
    range_color=(8, 70),
    scope='south america'
)

# fig.update_geos(fitbounds = "locations", visible = False)
fig.show()